# AI Tensorflow Project #1
Build and test a convolutional neural network for Traffic Sign Recognition using tensorflow

# Import tensorflow and Traffic sign data

In [14]:
# Load pickled data
import pickle
import tensorflow as tf
from sklearn.utils import shuffle
# TODO: Fill this in based on where you saved the training and testing data

training_file = "train.p"
validation_file="valid.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


X_train=X_train.reshape([X_train.shape[0],-1])
X_valid=X_valid.reshape([X_valid.shape[0],-1])
X_test=X_test.reshape([X_test.shape[0],-1])

X_train = (X_train/127.5-1.0)
X_valid = (X_valid/127.5-1.0)
X_test = (X_test/127.5-1.0)
print (X_train.shape, y_train.shape)
print (X_valid.shape, y_valid.shape)
print (X_test.shape, y_test.shape)

num_samples=X_train.shape[0]

(34799, 3072) (34799,)
(4410, 3072) (4410,)
(12630, 3072) (12630,)


# Define input/output

In [15]:
# MNIST data input (img shape: 32*32)
num_input = 32*32*3
# Traffic sign total classes 
num_classes = 43

# placeholders for input and output
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.int32, [None])

# Define layers parameters

In [16]:
# Define size of neurons per layer of 4 connected nerual layers
# Try change the number of layers and neurons per layer!
n_neurons = {
    1: 512,
    2: 256,
    3: 256,
    4: 128,
}
# Define the weight shape for future invoke
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_neurons[1]])),
    'h2': tf.Variable(tf.random_normal([n_neurons[1], n_neurons[2]])),
    'h3': tf.Variable(tf.random_normal([n_neurons[2], n_neurons[3]])),
    'h4': tf.Variable(tf.random_normal([n_neurons[3], n_neurons[4]]))
}
# define bias shape
biases = {
    'b1': tf.Variable(tf.random_normal([n_neurons[1]])),
    'b2': tf.Variable(tf.random_normal([n_neurons[2]])),
    'b3': tf.Variable(tf.random_normal([n_neurons[3]])),
    'b4': tf.Variable(tf.random_normal([n_neurons[4]])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Hyperparameters!!

In [17]:
# Parameters and model hyper-paramters
learning_rate = 0.0005 #0.001
#num_steps = 1000
batch_size = 128
display_step = 10
EPOCHS=3


In [21]:
def gen_batch(X, y, num_examples, batch_size):
    for offset in range(0, num_examples, batch_size):
        end = offset + batch_size
        if(end<=num_examples):
            batch_x, batch_y = X[offset:end], y[offset:end]
            yield (batch_x, batch_y)

# Define evaluation method without core neural network yet

In [22]:

def evaluate(logits):
    prediction = tf.nn.softmax(logits)
    one_hot_Y = tf.one_hot(Y, 43)
    cost_function = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_Y))
    
    # Check out other optimzers, basically compared to GD or SGD, Adam 
    # introduced momentum to (1) more effectively train (2) protect from local
    # minimal better; and adaptively adjust learning speed more effectively to
    # converge faster, than gradient descent
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(cost_function)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(one_hot_Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    # Start training
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver=tf.train.Saver()
        avg_acc=[]
        avg_loss=[]
        
        for iter in range(EPOCHS):
            total_acc=[]
            total_loss=[]
            step=0
            #offset=0
            num_steps=int(num_samples/batch_size)
            i_X_train, i_y_train=shuffle( X_train, y_train)    
            
            #for step in range(num_steps):
            for batch_x, batch_y in gen_batch(i_X_train, i_y_train, num_samples, batch_size):
                step+=1
                #end=offset+batch_size
                #batch_x, batch_y = i_X_train1[offset:end], i_y_train[offset:end]
                #offset+=batch_size
                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
                # Calculate batch loss and accuracy
                if (step % display_step == 0 or step == 1):
                    loss, acc = sess.run([cost_function, accuracy],
                                         feed_dict={X: batch_x, Y: batch_y})
                    print("EPOCHS" + str(iter) + ", Step " + str(step) \
                          + ", Loss= " + "{:.4f}".format(loss) \
                          + ", Training Accuracy= " + "{:.3f}".format(acc))
                    total_acc.append(acc)
                    total_loss.append(loss)
            avg_acc.append(np.mean(total_acc))
            avg_loss.append(np.mean(total_loss))
        print("Optimization Finished!")
        saver.save(sess, './cnn')

# Calculate accuracy for  train/valid/test images

        print("Training Accuracy:", \
            sess.run(accuracy, feed_dict={X: X_train,
                                          Y: y_train}))

        print("Valid Accuracy:", \
            sess.run(accuracy, feed_dict={X: X_valid,
                                          Y: y_valid}))
        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: X_test,
                                          Y: y_test}))
        

# Vanilla NN
A 4 hidden layer neural network <br>
L1: 784, 512 <br>
L2: 512, 256 <br>
L3: 256, 256 <br>
L4: 256, 128 <br>
Output: 128, 10 <br>

In [24]:
EPOCHS=50
def neural_net(x, n_layer):
    print("------Vanilla NN with {0} layers------".format(n_layer))
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    hidden_layers = {
        1: layer_1,
        2: layer_2,
        3: layer_3,
        4: layer_4
    }
    out_weight = tf.Variable(tf.random_normal([n_neurons[n_layer], num_classes]))
    out_layer = tf.matmul(hidden_layers[n_layer], out_weight) + biases['out']
    return out_layer
print(y_train.shape)
evaluate(neural_net(X, 4))

(34799,)
------Vanilla NN with 4 layers------
EPOCHS0, Step 1, Loss= 3485367.2500, Training Accuracy= 0.023
EPOCHS0, Step 2, Loss= 3542101.5000, Training Accuracy= 0.008
EPOCHS0, Step 3, Loss= 3535327.7500, Training Accuracy= 0.008
EPOCHS0, Step 4, Loss= 3145212.2500, Training Accuracy= 0.016
EPOCHS0, Step 5, Loss= 3052029.2500, Training Accuracy= 0.031
EPOCHS0, Step 6, Loss= 2891660.5000, Training Accuracy= 0.039
EPOCHS0, Step 7, Loss= 3061520.5000, Training Accuracy= 0.031
EPOCHS0, Step 8, Loss= 2574049.7500, Training Accuracy= 0.031
EPOCHS0, Step 9, Loss= 2825998.0000, Training Accuracy= 0.023
EPOCHS0, Step 10, Loss= 2637055.5000, Training Accuracy= 0.047
EPOCHS0, Step 11, Loss= 2527873.7500, Training Accuracy= 0.070
EPOCHS0, Step 12, Loss= 2432978.0000, Training Accuracy= 0.031
EPOCHS0, Step 13, Loss= 2320111.5000, Training Accuracy= 0.031
EPOCHS0, Step 14, Loss= 2551634.0000, Training Accuracy= 0.078
EPOCHS0, Step 15, Loss= 2173786.0000, Training Accuracy= 0.039
EPOCHS0, Step 16, 